# Statistical Downscaling and Bias-Adjustment - Advanced tools

The previous notebook covered the most common utilities of `xclim.sdba` for conventional cases. Here, we explore more advanced usage of `xclim.sdba` tools.

## Optimization with dask

Adjustment processes can be very heavy when we need to compute them over large regions and long timeseries. Using small groupings (like `time.dayofyear`) adds precision and robustness, but also decouples the load and computing complexity. Fortunately, unlike the heroic pioneers of scientific computing who managed to write parallelized FORTRAN, we now have [dask](https://dask.org/). With only a few parameters, we can magically distribute the computing load to multiple workers and threads.

A good first read on the use of dask within xarray are the latter's [Optimization tips](https://xarray.pydata.org/en/stable/user-guide/dask.html#optimization-tips).

Some `xclim.sdba`-specific tips:

* Most adjustment method will need to perform operation on the whole `time` coordinate, so it is best to optimize chunking along the other dimensions. This is often different from how public data is shared, where more universal 3D chunks are used.

    Chunking of outputs can be controlled in xarray's [to_netcdf](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.to_netcdf.html?highlight=to_netcdf#xarray.Dataset.to_netcdf). We also suggest using [Zarr](https://zarr.readthedocs.io/en/stable/) files. According to [its creators](https://ui.adsabs.harvard.edu/abs/2018AGUFMIN33A..06A/abstract), `zarr` stores should give better performances, especially because of their better ability for parallel I/O. See [Dataset.to_zarr](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.to_zarr.html?highlight=to_zarr#xarray.Dataset.to_zarr) and this useful [rechunking package](https://rechunker.readthedocs.io).


* One of the main bottleneck for adjustments with small groups is that dask needs to build and optimize an enormous task graph. This issue has been greatly reduced with xclim 0.27 and the use of `map_blocks` in the adjustment methods. However, not all adjustment methods use this optimized syntax.

    In order to help dask, one can split the processing in parts. For splitting training and adjustment, see [the section below](#Initializing-an-Adjustment-object-from-a-training-dataset).


* Another massive bottleneck of parallelization of `xarray` is the thread-locking behaviour of some methods. It is quite difficult to isolate and avoid these locking instances, so one of the best workarounds is to use `dask` configurations with many _processes_ and few _threads_. The former do not share memory and thus are not impacted when a lock is activated from a thread in another worker. However, this adds many memory transfer operations and, by experience, reduces dask's ability to parallelize some pipelines. Such a dask Client is usually created with a large `n_workers` and a small `threads_per_worker`.


* Sometimes, datasets have auxiliary coordinates (for example : lat / lon in a rotated pole dataset). Xarray handles these variables as data variables and will **not** load them if dask is used. However, in some operations, `xclim` or `xarray` will trigger access to those variables, triggering computations each time, since they are `dask`-based. To avoid this behaviour, one can load the coordinates, or simply remove them from the inputs.

## LOESS smoothing and detrending

As described in Cleveland (1979), locally weighted linear regressions are multiple regression methods using a nearest-neighbour approach. Instead of using all data points to compute a linear or polynomial regression, LOESS algorithms compute a local regression for each point in the dataset, using only the k-nearest neighbours as selected by a weighting function. This weighting function must fulfill some strict requirements, see the doc of `xclim.sdba.loess.loess_smoothing` for more details.

In xclim's implementation, the user can choose between local _constancy_ ($d=0$, local estimates are weighted averages) and local _linearity_ ($d=1$, local estimates are taken from linear regressions). Two weighting functions are currently implemented : "tricube" ($w(x) = (1 - x^3)^3$) and "gaussian" ($w(x) = e^{-x^2 / 2\sigma^2}$). Finally, the number of Cleveland's _robustifying iterations_ is controllable through `niter`. After computing an estimate of $y(x)$, the weights are modulated by a function of the distance between the estimate and the points and the procedure is started over. These iterations are made to weaken the effect of outliers on the estimate.

The next example shows the application of the LOESS to daily temperature data. The black line and dot are the estimated $y$, outputs of the `sdba.loess.loess_smoothing` function, using local linear regression (passing $d = 1$), a window spanning 20% ($f = 0.2$) of the domain, the "tricube" weighting function and only one iteration. The red curve illustrates the weighting function on January 1st 2014, where the red circles are the nearest-neighbours used in the estimation.

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import nc_time_axis
import numpy as np
import xarray as xr

from xclim.sdba import loess

%matplotlib inline

In [ ]:
# Daily temperature data from xarray's tutorials
ds = xr.tutorial.open_dataset("air_temperature").resample(time="D").mean()
tas = ds.isel(lat=0, lon=0).air

# Compute the smoothed series
f = 0.2
ys = loess.loess_smoothing(tas, d=1, weights="tricube", f=f, niter=1)

# Plot data points and smoothed series
fig, ax = plt.subplots()
ax.plot(tas.time, tas, "o", fillstyle="none")
ax.plot(tas.time, ys, "k")
ax.set_xlabel("Time")
ax.set_ylabel("Temperature [K]")

## The code below calls internal functions to demonstrate how the weights are computed.

# LOESS algorithms as implemented here use scaled coordinates.
x = tas.time
x = (x - x[0]) / (x[-1] - x[0])
xi = x[366]
ti = tas.time[366]

# Weighting function take the distance with all neighbors scaled by the r parameter as input
r = int(f * tas.time.size)
h = np.sort(np.abs(x - xi))[r]
weights = loess._tricube_weighting(np.abs(x - xi).values / h)

# Plot nearest neighbors and weighing function
wax = ax.twinx()
wax.plot(tas.time, weights, color="indianred")
ax.plot(
    tas.time, tas.where(tas * weights > 0), "o", color="lightcoral", fillstyle="none"
)

ax.plot(ti, ys[366], "ko")
wax.set_ylabel("Weights")
plt.show()

LOESS smoothing can suffer from heavy boundary effects. On the previous graph, we can associate the strange bend on the left end of the line to them. The next example shows a stronger case. Usually, $\frac{f}{2}N$ points on each side should be discarded. On the other hand, LOESS has the advantage of always staying within the bounds of the data.


### LOESS Detrending

In climate science, it can be used in the detrending process. `xclim` provides `sdba.detrending.LoessDetrend` in order to compute trend with the LOESS smoothing and remove them from timeseries.

First we create some toy data with a sinusoidal annual cycle, random noise and a linear temperature increase.

In [ ]:
time = xr.cftime_range("1990-01-01", "2049-12-31", calendar="noleap")
tas = xr.DataArray(
    (
        10 * np.sin(time.dayofyear * 2 * np.pi / 365)
        + 5 * (np.random.random_sample(time.size) - 0.5)  # Annual variability
        + np.linspace(0, 1.5, num=time.size)  # Random noise
    ),  # 1.5 degC increase in 60 years
    dims=("time",),
    coords={"time": time},
    attrs={"units": "degC"},
    name="temperature",
)
tas.plot()

Then we compute the trend on the data. Here, we compute on the whole timeseries (`group='time'`) with the parameters suggested above.

In [ ]:
from xclim.sdba.detrending import LoessDetrend

# Create the detrending object
det = LoessDetrend(group="time", d=0, niter=2, f=0.2)
# Fitting returns a new object and computes the trend.
fit = det.fit(tas)
# Get the detrended series
tas_det = fit.detrend(tas)

In [ ]:
fig, ax = plt.subplots()
fit.ds.trend.plot(ax=ax, label="Computed trend")
ax.plot(time, np.linspace(0, 1.5, num=time.size), label="Expected tred")
ax.plot([time[0], time[int(0.1 * time.size)]], [0.4, 0.4], linewidth=6, color="gray")
ax.plot([time[-int(0.1 * time.size)], time[-1]], [1.1, 1.1], linewidth=6, color="gray")
ax.legend()

As said earlier, this example shows how the Loess has strong boundary effects. It is recommended to remove the $\frac{f}{2}\cdot N$ outermost points on each side, as shown by the  gray bars in the graph above.

## Initializing an Adjustment object from a training dataset

For large scale uses, when the training step deserves its own computation and write to disk, or simply when there are multiples `sim` to be adjusted with the same training, it is helpful to be able to instantiate the Adjustment objects from the training dataset itself. This trick relies on a global attribute "adj_params" set on the training dataset.

In [ ]:
import numpy as np
import xarray as xr

# Create toy data for the example, here fake temperature timeseries
t = xr.cftime_range("2000-01-01", "2030-12-31", freq="D", calendar="noleap")
ref = xr.DataArray(
    (
        -20 * np.cos(2 * np.pi * t.dayofyear / 365)
        + 2 * np.random.random_sample((t.size,))
        + 273.15
        + 0.1 * (t - t[0]).days / 365
    ),  # "warming" of 1K per decade,
    dims=("time",),
    coords={"time": t},
    attrs={"units": "K"},
)
sim = xr.DataArray(
    (
        -18 * np.cos(2 * np.pi * t.dayofyear / 365)
        + 2 * np.random.random_sample((t.size,))
        + 273.15
        + 0.11 * (t - t[0]).days / 365
    ),  # "warming" of 1.1K per decade
    dims=("time",),
    coords={"time": t},
    attrs={"units": "K"},
)

ref = ref.sel(time=slice(None, "2015-01-01"))
hist = sim.sel(time=slice(None, "2015-01-01"))

In [ ]:
from xclim.sdba.adjustment import QuantileDeltaMapping

QDM = QuantileDeltaMapping.train(
    ref, hist, nquantiles=15, kind="+", group="time.dayofyear"
)
QDM

The trained `QDM` exposes the training data in the `ds` attribute, Here, we will write it to disk, read it back and initialize a new object from it. Notice the `adj_params` in the dataset, that has the same value as the repr string printed just above. Also, notice the `_xclim_adjustment` attribute that contains a JSON string, so we can rebuild the adjustment object later.

In [ ]:
QDM.ds

In [ ]:
QDM.ds.to_netcdf("QDM_training.nc")
ds = xr.open_dataset("QDM_training.nc")
QDM2 = QuantileDeltaMapping.from_dataset(ds)
QDM2

In the case above, creating a full object from the dataset doesn't make the most sense since we are in the same python session, with the "old" object still available. This method effective when we reload the training data in a different python session, say on another computer. **However, take note that there is no retro-compatibility insurance.** If the `QuantileDeltaMapping` class was to change in a new xclim version, one would not be able to create the new object from a dataset saved with the old one.

For the case where we stay in the same python session, it is still useful to trigger the dask computations. For small datasets, that could mean a simple `QDM.ds.load()`, but sometimes even the training data is too large to be full loaded in memory. In that case, we could also do:

In [ ]:
QDM.ds.to_netcdf("QDM_training2.nc")
ds = xr.open_dataset("QDM_training2.nc")
ds.attrs["title"] = "This is the dataset, but read from disk."
QDM.set_dataset(ds)
QDM.ds

In [ ]:
QDM2.adjust(sim)

## Retrieving extra output diagnostics

To fully understand what is happening during the bias-adjustment process, sdba can output _diagnostic_ variables, giving more visibility to what the adjustment is doing behind the scene. This behaviour, a `verbose` option, is controlled by the `sdba_extra_output` option, set with `xclim.set_options`. When `True`, `train` calls are instructed to include additional variables to the training datasets. In addition, the `adjust` calls will always output a dataset, with `scen` and, depending on the algorithm, other diagnostics variables. See the documentation of each `Adjustment` objects to see what extra variables are available.

For the moment, this feature is still under construction and only a few `Adjustment` actually provide these extra outputs. Please open issues on the GitHub repo if you have needs or ideas of interesting diagnostic variables.

For example, `QDM.adjust` adds `sim_q`, which gives the quantile of each element of `sim` within its group.

In [ ]:
from xclim import set_options

with set_options(sdba_extra_output=True):
    QDM = QuantileDeltaMapping.train(
        ref, hist, nquantiles=15, kind="+", group="time.dayofyear"
    )
    out = QDM.adjust(sim)

out.sim_q

## Moving window for adjustments

Some Adjustment methods require that the adjusted data (`sim`) be of the same length (same number of points) than the training data (`ref` and `hist`). These requirements often ensure conservation of statistical properties and a better representation of the climate change signal over the long adjusted timeseries.

In opposition to a conventional "rolling window", here it is the _years_ that are the base units of the window, not the elements themselves. `xclim` implements `sdba.construct_moving_yearly_window` and `sdba.unpack_moving_yearly_window` to manipulate data in that goal. The "construct" function cuts the data in overlapping windows of a certain length (in years) and stacks them along a new `"movingdim"` dimension, alike to xarray's `da.rolling(time=win).construct('movingdim')`, but with yearly steps. The step between each window can also be controlled. This argument is an indicator of how many years overlap between each window. With a value of 1 (the default), a window will have `window - 1` years overlapping with the previous one. `step = window` will result in no overlap at all.

By default, the result is chunked along this `'movingdim'` dimension. For this reason, the method is expected to be more computationally efficient (when using `dask`) than looping over the windows.

Note that this results in two restrictions:

1. The constructed array has the same "time" axis for all windows. This is a problem if the actual _year_ is of importance for the adjustment, but this is not the case for any of xclim's current adjustment methods.
2. The input timeseries must be in a calendar with uniform year lengths. For daily data, this means only the "360_day", "noleap" and "all_leap" calendars are supported.

The "unpack" function does the opposite : it concatenates the windows together to recreate the original timeseries.
The time points that are not part of a window will not appear in the reconstructed timeseries.
If `append_ends` is True, the reconstructed timeseries will go from the first time point of the first window to the last time point of the last window. In the middle, the central `step` years are kept from each window.
If `append_ends` is False, only the central `step` years are kept from each window. Which means the final timeseries has  `(window - step) / 2` years missing on either side, with the extra year missing on the right in case of an odd `(window - step)`. We are missing data, but the contribution from each window is equal.

Here, as `ref` and `hist` cover 15 years, we will use a window of 15 on sim. With a step of two (2), this means the first window goes from 2000 to 2014 (inclusive). The last window goes from 2016 to 2030. `window - step = 13`, so six (6) years will be missing at the beginning of the final `scen` and seven (7) years at the end.

In [ ]:
QDM = QuantileDeltaMapping.train(
    ref, hist, nquantiles=15, kind="+", group="time.dayofyear"
)

scen_nowin = QDM.adjust(sim)

In [ ]:
sim

In [ ]:
from xclim.sdba import construct_moving_yearly_window, unpack_moving_yearly_window

sim_win = construct_moving_yearly_window(sim, window=15, step=2)
sim_win

Here, we retrieve the full timeseries.

In [ ]:
scen_win = unpack_moving_yearly_window(QDM.adjust(sim_win), append_ends=True)
scen_win

Whereas here, we have gaps at the edges.

In [ ]:
scen_win = unpack_moving_yearly_window(QDM.adjust(sim_win), append_ends=False)
scen_win

Here is another short example, with an uneven number of years. Here `sim` goes from 2000 to 2029 (30 years instead of 31). With a step of 2 and a window of 15, the first window goes again from 2000 to 2014, but the last one is now from 2014 to 2028. The next window would be 2016-2030, but that last year doesn't exist.

In [ ]:
sim_win = construct_moving_yearly_window(
    sim.sel(time=slice("2000", "2029")), window=15, step=2
)
sim_win

Here, we don't recover the full timeseries, even when we append the ends, because 2029 is not part of a window.

In [ ]:
sim2 = unpack_moving_yearly_window(sim_win, append_ends=True)
sim2

Without appending the ends, the final timeseries is from 2006 to 2021, 6 years missing at the beginning, like last time and **8** years missing at the end.

In [ ]:
sim2 = unpack_moving_yearly_window(sim_win, append_ends=False)
sim2

## Full example: Multivariate adjustment in the additive space

The following example shows a complete bias-adjustment workflow using the `PrincipalComponents` method in a multi-variate configuration. Moreover, it uses the trick showed by [Alavoine et Grenier (2022)](https://doi.org/10.31223/X5C34C) to transform "multiplicative" variable to the "additive" space using log and logit transformations. This way, we can perform multi-variate adjustment with variables that couldn't be used in the same _kind_ of adjustment, like "tas" and "hurs".

We will transform the variables that need it to the additive space, adding some jitter in the process to avoid $log(0)$ situations. Then, we will stack the different variables into a single `DataArray`, allowing us to use `PrincipalComponents` in a multi-variate way. Following the PCA, a simple quantile-mapping method is used, both adjustment acting on the residuals, while the mean of the simulated trend is adjusted on its own. Each step will be explained.

First, open the data, convert the calendar and the units. Because we will perform adjustments on "dayofyear" groups (with a window), keeping standard calendars results in an extra "dayofyear" with only a quarter of the data. It's usual to transform to a "noleap" calendar, which drops the 29th of February, as it only has a small impact on the data.

In [ ]:
import xclim.sdba as sdba
from xclim.core.calendar import convert_calendar
from xclim.core.units import convert_units_to
from xclim.testing import open_dataset

group = sdba.Grouper("time.dayofyear", window=31)

dref = convert_calendar(open_dataset("sdba/ahccd_1950-2013.nc"), "noleap").sel(
    time=slice("1981", "2010")
)
dsim = open_dataset("sdba/CanESM2_1950-2100.nc")

dref = dref.assign(
    tasmax=convert_units_to(dref.tasmax, "K"),
)
dsim = dsim.assign(pr=convert_units_to(dsim.pr, "mm/d"))

### 1. Jitter, additive space transformation and variable stacking
Here, `tasmax` is already ready to be adjusted in an additive way, because all data points are far from the physical zero (0 K). This is not the case for `pr`, which is why we want to transform that variable to the additive space, to avoid splitting our workflow in two. For `pr` the "log" transformation is simply:

$$ pr' = \ln\left(pr - b\right) $$

Where $b$ is the lower bound, here 0 mm/d. However, we could have exact zeros (0 mm/d) in the datasets, which will translate into $-\infty$. To avoid this, we simply replace the smallest values by a random distribution of very small, but not problematic, values. In the following, all values below 0.1 mm/d are replaced by a uniform random distribution of values within the range (0, 0.1) mm/d (bounds excluded).

Finally, the variables are stacked together into a single `DataArray`.

In [ ]:
dref_as = dref.assign(
    pr=sdba.processing.to_additive_space(
        sdba.processing.jitter(dref.pr, lower="0.1 mm/d", minimum="0 mm/d"),
        lower_bound="0 mm/d",
        trans="log",
    )
)
ref = sdba.stack_variables(dref_as)

dsim_as = dsim.assign(
    pr=sdba.processing.to_additive_space(
        sdba.processing.jitter(dsim.pr, lower="0.1 mm/d", minimum="0 mm/d"),
        lower_bound="0 mm/d",
        trans="log",
    )
)
sim = sdba.stack_variables(dsim_as)
sim

### 2. Get residuals and trends
The adjustment will be performed on residuals only. The adjusted timeseries `sim` will be detrended with the LOESS routine described above. Because of the short length of  `ref` and `hist` and the potential boundary effects of using LOESS with them, we compute the 30-year mean. In other words, instead of _detrending_ inputs, we are _normalizing_ those inputs.

While the residuals are adjusted with `PrincipalComponents` and `EmpiricalQuantileMapping`, the trend of  `sim` still needs to be offset according to the means of `ref` and `hist`. This is similar to what `DetrendedQuantileMapping` does. The offset step could have been done on the trend itself or at the end on `scen`, it doesn't really matter. We do it here because it keeps it close to where the `scaling` is computed.

In [ ]:
ref_res, ref_norm = sdba.processing.normalize(ref, group=group, kind="+")
hist_res, hist_norm = sdba.processing.normalize(
    sim.sel(time=slice("1981", "2010")), group=group, kind="+"
)
scaling = sdba.utils.get_correction(hist_norm, ref_norm, kind="+")

In [ ]:
sim_scaled = sdba.utils.apply_correction(
    sim, sdba.utils.broadcast(scaling, sim, group=group), kind="+"
)

loess = sdba.detrending.LoessDetrend(group=group, f=0.2, d=0, kind="+", niter=1)
simfit = loess.fit(sim_scaled)
sim_res = simfit.detrend(sim_scaled)

### 3. Adjustments
Following, Alavoine et Grenier (2022), we decided to perform the multivariate Principal Components adjustment first and then re-adjust with the simple Quantile-Mapping.

In [ ]:
PCA = sdba.adjustment.PrincipalComponents.train(
    ref_res, hist_res, group=group, crd_dim="multivar", best_orientation="simple"
)

scen1_res = PCA.adjust(sim_res)

In [ ]:
EQM = sdba.adjustment.EmpiricalQuantileMapping.train(
    ref_res,
    scen1_res.sel(time=slice("1981", "2010")),
    group=group,
    nquantiles=50,
    kind="+",
)

scen2_res = EQM.adjust(scen1_res, interp="linear", extrapolation="constant")

### 4. Re-trend and transform back to the physical space
Add back the trend (which includes the scaling), unstack the variables to a dataset and transform `pr` back to the physical space. All functions have conserved and handled the attributes, so we don't need to repeat the additive space bounds. The annual cycle of both variables on the reference period in Vancouver is plotted to confirm the adjustment adds a positive effect.

In [ ]:
scen = simfit.retrend(scen2_res)
dscen_as = sdba.unstack_variables(scen)
dscen = dscen_as.assign(pr=sdba.processing.from_additive_space(dscen_as.pr))

In [ ]:
dref.tasmax.sel(time=slice("1981", "2010"), location="Vancouver").groupby(
    "time.dayofyear"
).mean().plot(label="obs")
dsim.tasmax.sel(time=slice("1981", "2010"), location="Vancouver").groupby(
    "time.dayofyear"
).mean().plot(label="raw")
dscen.tasmax.sel(time=slice("1981", "2010"), location="Vancouver").groupby(
    "time.dayofyear"
).mean().plot(label="scen")
plt.legend()

In [ ]:
dref.pr.sel(time=slice("1981", "2010"), location="Vancouver").groupby(
    "time.dayofyear"
).mean().plot(label="obs")
dsim.pr.sel(time=slice("1981", "2010"), location="Vancouver").groupby(
    "time.dayofyear"
).mean().plot(label="raw")
dscen.pr.sel(time=slice("1981", "2010"), location="Vancouver").groupby(
    "time.dayofyear"
).mean().plot(label="scen")
plt.legend()

## Tests for sdba

It can be useful to perform diagnostic tests on adjusted simulations to assess if the bias correction method is working properly, or to compare two different bias correction techniques.

A diagnostic test includes calculations of a property (mean, 20-year return value, annual cycle amplitude, ...) on the simulation and on the scenario (adjusted simulation), then a measure (bias, relative bias, ratio, ...) of the  difference. Usually, the property collapse the time dimension of the simulation/scenario and returns one value by grid point.

You'll find those in ``xclim.sdba.properties`` and ``xclim.sdba.measures``, where they are implemented as special subclasses of xclim's ``Indicator``, which means they can be worked with the same way as conventional indicators (used in YAML modules for example).

In [ ]:
from matplotlib import pyplot as plt

import xclim as xc
from xclim import sdba
from xclim.testing import open_dataset

# load test data
hist = open_dataset("sdba/CanESM2_1950-2100.nc").sel(time=slice("1950", "1980")).tasmax
ref = open_dataset("sdba/nrcan_1950-2013.nc").sel(time=slice("1950", "1980")).tasmax
sim = (
    open_dataset("sdba/CanESM2_1950-2100.nc").sel(time=slice("1980", "2010")).tasmax
)  # biased

# learn the bias in historical simulation compared to reference
QM = sdba.EmpiricalQuantileMapping.train(
    ref, hist, nquantiles=50, group="time", kind="+"
)

# correct the bias in the future
scen = QM.adjust(sim, extrapolation="constant", interp="nearest")
ref_future = (
    open_dataset("sdba/nrcan_1950-2013.nc").sel(time=slice("1980", "2010")).tasmax
)  # truth

plt.figure(figsize=(15, 5))
lw = 0.3
sim.isel(location=1).plot(label="sim", linewidth=lw)
scen.isel(location=1).plot(label="scen", linewidth=lw)
hist.isel(location=1).plot(label="hist", linewidth=lw)
ref.isel(location=1).plot(label="ref", linewidth=lw)
ref_future.isel(location=1).plot(label="ref_future", linewidth=lw)
leg = plt.legend()
for legobj in leg.legendHandles:
    legobj.set_linewidth(2.0)

In [ ]:
# calculate the mean warm Spell Length Distribution
sim_prop = sdba.properties.spell_length_distribution(
    da=sim, thresh="28 degC", op=">", stat="mean", group="time"
)


scen_prop = sdba.properties.spell_length_distribution(
    da=scen, thresh="28 degC", op=">", stat="mean", group="time"
)

ref_prop = sdba.properties.spell_length_distribution(
    da=ref_future, thresh="28 degC", op=">", stat="mean", group="time"
)
# measure the difference between the prediction and the reference with an absolute bias of the properties
measure_sim = sdba.measures.bias(sim_prop, ref_prop)
measure_scen = sdba.measures.bias(scen_prop, ref_prop)

plt.figure(figsize=(5, 3))
plt.plot(measure_sim.location, measure_sim.values, ".", label="biased model (sim)")
plt.plot(measure_scen.location, measure_scen.values, ".", label="adjusted model (scen)")
plt.title(
    "Bias of the mean of the warm spell \n length distribution compared to observations"
)
plt.legend()
plt.ylim(-2.5, 2.5)

It is possible the change the 'group' of the property from 'time' to 'time.season' or 'time.month'.
 This will return 4 or 12 values per grid point, respectively.

In [ ]:
# calculate the mean warm Spell Length Distribution
sim_prop = sdba.properties.spell_length_distribution(
    da=sim, thresh="28 degC", op=">", stat="mean", group="time.season"
)

scen_prop = sdba.properties.spell_length_distribution(
    da=scen, thresh="28 degC", op=">", stat="mean", group="time.season"
)

ref_prop = sdba.properties.spell_length_distribution(
    da=ref_future, thresh="28 degC", op=">", stat="mean", group="time.season"
)
# Properties are often associated with the same measures. This correspondance is implemented in xclim:
measure = sdba.properties.spell_length_distribution.get_measure()
measure_sim = measure(sim_prop, ref_prop)
measure_scen = measure(scen_prop, ref_prop)

fig, axs = plt.subplots(2, 2, figsize=(9, 6))
axs = axs.ravel()
for i in range(4):
    axs[i].plot(
        measure_sim.location, measure_sim.values[:, i], ".", label="biased model (sim)"
    )
    axs[i].plot(
        measure_scen.location,
        measure_scen.isel(season=i).values,
        ".",
        label="adjusted model (scen)",
    )
    axs[i].set_title(measure_scen.season.values[i])
    axs[i].legend(loc="lower right")
    axs[i].set_ylim(-2.5, 2.5)
fig.suptitle(
    "Bias of the mean of the warm spell length distribution compared to observations"
)
plt.tight_layout()
